In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random, shutil, time

import warnings
warnings.filterwarnings('ignore')

import sys

In [3]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/0_before_reformat/LPR/merge data


In [4]:
# mac - phase I && II - LPR - tall - before_Lisa_fix
file_path_before_fix_full = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/before_Lisa_fix/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall.csv'
df_before_fix_full = pd.read_csv(file_path_before_fix_full, header = 0)
df_before_fix_full['chemical.id'] = df_before_fix_full['chemical.id'].astype(str)
# 344 unique chemical IDs

#unique_chemical_IDs = np.unique(df_before_fix_full['chemical.id'])
#display (len(unique_chemical_IDs)) 

display(df_before_fix_full.head())

,chemical.id,bottle.id,conc,plate.id,well,endpoint,value
0,2752,TX000657,0.64,TP972-H6-P1,B3,t3,0.4
1,2752,TX000657,0.64,TP972-H6-P1,F3,t3,0.0
2,2752,TX000657,6.40,TP972-H6-P1,B8,t3,37.5
3,2752,TX000657,0.64,TP972-H6-P1,C3,t3,0.0
4,2752,TX000657,0.64,TP972-H6-P1,F3,t3,0.0


In [10]:
# mac - phase I && II - LPR - tall - after_Lisa_fix
file_path_after_fix_149 = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/partial/uM_removed/149_Toxcast_zf_LPR_data_phase_2_2021APR09.csv'
df_after_fix_149 = pd.read_csv(file_path_after_fix_149, header = 0)
df_after_fix_149['chemical.id'] = df_after_fix_149['chemical.id'].astype(str)
# 149 unique chemical IDs

unique_chemical_IDs = np.unique(df_after_fix_149['chemical.id'])
display (len(unique_chemical_IDs)) 
#display(df_after_fix_149.head())

# not working
#df_after_fix_149 = df_after_fix_149.replace(to_replace = " uM", value = "")
#display(df_after_fix_149.head())

#df_after_fix_per_chemical = df_after_fix_149.loc[df_after_fix_149['chemical.id'] == '2752',:]
#display (df_after_fix_per_chemical.head())

#df_after_fix_per_chemical = df_after_fix_per_chemical

149

In [11]:
start_time = time.time()

def merge(chemical_index, df_before_fix_full, df_after_fix_149, df_merged):
    df_before_fix_per_chemical = df_before_fix_full.loc[df_before_fix_full['chemical.id'] == str(chemical_index),:]
    #display ('before_fix:', df_before_fix_per_chemical.tail(1))
    
    df_after_fix_per_chemical  = df_after_fix_149.loc[df_after_fix_149['chemical.id'] == str(chemical_index),:]
    #display ("after_fix:", df_after_fix_per_chemical.tail(1))
    
    if (df_after_fix_per_chemical.empty == False):
        print ("replaced with fixed one")
        df_merged = pd.concat([df_merged, df_after_fix_per_chemical])
    else:
        df_merged = pd.concat([df_merged, df_before_fix_per_chemical])
        
    #display ("df_merged:", df_merged.tail(1))
        
    return df_merged
########### end of def merge()

df_merged = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_before_fix_full['chemical.id'])
if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 2)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#all_chemical_id_s = [2752]

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("\nchemical_index:" + str(chemical_index))
    number_of_chemicals_processed += 1
    df_merged = merge(chemical_index, df_before_fix_full, df_after_fix_149, df_merged)
    write_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(write_this)

df_merged.to_csv("merged.csv", index=False)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) 
# took 12 minutes for 335 chemicals


chemical_index:1030
1 chemicals processed out of 344

chemical_index:1031
2 chemicals processed out of 344

chemical_index:1105
3 chemicals processed out of 344

chemical_index:1114
4 chemicals processed out of 344

chemical_index:1119
replaced with fixed one
5 chemicals processed out of 344

chemical_index:1161
6 chemicals processed out of 344

chemical_index:1163
7 chemicals processed out of 344

chemical_index:1166
8 chemicals processed out of 344

chemical_index:1167
replaced with fixed one
9 chemicals processed out of 344

chemical_index:1171
10 chemicals processed out of 344

chemical_index:1172
11 chemicals processed out of 344

chemical_index:1174
12 chemicals processed out of 344

chemical_index:1175
13 chemicals processed out of 344

chemical_index:1176
14 chemicals processed out of 344

chemical_index:1177
15 chemicals processed out of 344

chemical_index:1178
16 chemicals processed out of 344

chemical_index:1179
17 chemicals processed out of 344

chemical_index:1180
18 ch

In [88]:
# this is not working,,,



def remove_uM(conc_index, df_merged, df_merged_uM_removed):
    display ('df_merged:', df_merged.tail(1))
    
    # df_before_fix_per_chemical = df_before_fix_full.loc[df_before_fix_full['chemical.id'] == str(chemical_index),:]
    df_merged_per_conc = df_merged.loc[df_merged["conc"] == str(conc_index),:]
    display ('before_fix:', df_merged_per_conc.tail(1))
    a=b
    display(np.unique(df_merged_per_conc))
        
    return df_merged_uM_removed
########### end of def remove_uM()


df_merged_uM_removed = pd.DataFrame()

#full_devel = "full"
full_devel = "devel"

all_conc = np.unique(df_merged['chemical.id'])
if (full_devel == "devel"):
    randomly_chosen = random.sample(set(all_conc), 2)
    all_conc = []
    for i in range(len(randomly_chosen)):
        all_conc.append(randomly_chosen[i])

for conc_index in all_conc:
    print("\n\nconc_index:" + str(conc_index))
    df_merged_uM_removed = remove_uM(conc_index, df_merged, df_merged_uM_removed)

merged.to_csv("merged.csv", index=False)

# df_merged.conc = df_merged.conc.replace(" uM", "")
    
#replaced = df_merged["conc"].str.replace("uM", "")
#    print (replaced)
#    if (replaced != "NA"):
#        df_merged["conc"] = replaced




conc_index:296


'df_merged:'

,chemical.id,bottle.id,conc,plate.id,well,endpoint,value
413183,1606,TX003387,0 uM,1,H6,t17,0.0


'before_fix:'

,chemical.id,bottle.id,conc,plate.id,well,endpoint,value


NameError: name 'b' is not defined

In [45]:
#all_chemical_id_s_merged = np.unique(str(merged['chemical.id']))
#all_chemical_id_s_merged = np.unique(merged['chemical.id'])
#display(all_chemical_id_s_merged)

array([  57,  332,  380,  778, 1161, 1187, 1614, 2686, 2877, 2893])